# Get Programming with F# by [Isaac Abraham](https://github.com/isaacabraham)

## “Fixing the billion-dollar mistake”

The post-2018 news for the handling of nulls in C# is that _null parameter checking_ (or _Parameter Null Checking_) is still [a proposal](https://github.com/dotnet/csharplang/blob/main/proposals/param-nullchecking.md): it did not make it to C# 10. This proposed feature is on top of other null-related features in C# such as:

- “`!` (null-forgiving) operator” [📖 [docs](https://docs.microsoft.com/en-us/dotnet/csharp/language-reference/operators/null-forgiving)]
- “Nullable reference types” [📖 [docs](https://docs.microsoft.com/en-us/dotnet/csharp/nullable-references)]
- “`??` and `??=` operators” [📖 [docs](https://docs.microsoft.com/en-us/dotnet/csharp/language-reference/operators/null-coalescing-operator)]
- “Null-conditional operators `?.` and `?[]`” [📖 [docs](https://docs.microsoft.com/en-us/dotnet/csharp/language-reference/operators/member-access-operators#null-conditional-operators--and-)]
- “Nullable value types” [📖 [docs](https://docs.microsoft.com/en-us/dotnet/csharp/language-reference/builtin-types/nullable-value-types)]

>Even members of the C# complier team have said that they regret putting null into the language. The problem is that now it’s in, it’s almost impossible to remove. …The simple truth is that the .NET type system itself isn’t geared toward allowing us to easily and consistently reason about mandatory and optional data.

In the world of C#, there is a project called _Optional_ [[GitHub](https://github.com/nlkl/Optional)] [📦 [NuGet](https://www.nuget.org/packages/Optional/)] led by [Nils Lück](https://www.linkedin.com/in/nils-l%C3%BCck-a2258358/?originalSubdomain=dk). The name of this C# project is clearly a direct reference to what is built from the ground up in F#: the `Option` type [📖 [docs](https://docs.microsoft.com/en-us/dotnet/fsharp/language-reference/options)].

@[BryanWilhite](https://twitter.com/BryanWilhite)


In [ ]:
#!about

.NET Interactive© 2020 Microsoft CorporationVersion: 1.0.255902+b0afbdc47dec91e62b0c5cb587a0f2c24242eca8Build date: 2021-11-09T18:27:50.0000000Zhttps://github.com/dotnet/interactive
